# Imports

In [1]:
%matplotlib qt5

import pyaccel
import pymodels
import numpy as np
import matplotlib.pyplot as plt
from mathphys.functions import save_pickle, load_pickle

# Utils

In [11]:
def set_vchamber_h(params, width):
    for iten in params.scraper_indices:
        params.fitm[iten].hmax = width
        params.fitm[iten].hmin = -width
    
    return None


def track_mchn_stdy(params, bunch, nturn, coord_idx, increment):
    # setting param to be the index of the array [x,x',y,y',delta,z]

    bunch[coord_idx] += increment
    tracked = pyaccel.tracking.ring_pass(params.fitm, particles=bunch,
                                         nr_turns=nturn, turn_by_turn= True,
                                         element_offset=params.nlk_index, parallel=False)
    
    part_out, lost_flag, turn_lost, index_lost, plane_lost = tracked
    # print(lost_flag)
    
    turnl_element = []

    for i,iten in enumerate(turn_lost):
        if iten == nturn and index_lost[i] == params.nlk_index: # ignora elétrons que não foram perdidos
            pass
        else:
            turnl_element.append((iten, index_lost[i]))
    
    return turnl_element

def varying_incmnts(params, bunch, nturn, coord_idx, coord_amp, coord_amp_nrpts, scraper_width):

    ''' 
        acc: accelerator 
        bunch: particle initial conditions
        par: index of [x,x',y,y',delta,z] -> [0,1,2,3,4,5]
        rng: range of the increment
        nturn: number of turns
        val: variable that goes in numpy linspace, val is the value till the variation must be performed
        strin: is a string, this string represents the fam_name of the scraper 'SHVC' or 'SVVC'
        height: is the value that the user desires for setting the height of vchamber
    '''

    lostpos_n = []
    lostpos_m = []
    losttur_n = []
    losttur_m = []

    inc = np.linspace(0, coord_amp, coord_amp_nrpts)
    incdif = np.diff(inc)

    inc_qlst_n = []
    inc_qlst_m = []

    # nominal model tracking simulation
    bunchi = bunch.copy()
    for j, iten in enumerate(incdif):
        track = track_mchn_stdy(params, bunchi, nturn=nturn, coord_idx=coord_idx, increment=iten)
        length = len(track)
        if len(track) == 0:
            pass
        else:
            mean = np.mean(bunchi[0])*1e3
            inc_qlst_n.append([mean,length]) # this will be the y axis of the graphic
            for lst in track:
                lost_turn, lost_pos = lst
                losttur_n.append([lost_turn, mean])
                lostpos_n.append([lost_pos, mean])

    # # changed scraper width within model tracking simulation
    set_vchamber_h(params, scraper_width) # after calling this function, vchamber's width will be changed
    print(params.fitm[params.scraper_indices[0]])
    
    bunchi = bunch.copy()
    for j, iten in enumerate(incdif):
        track = track_mchn_stdy(params, bunchi, nturn=nturn, coord_idx=coord_idx, increment=iten)
        length = len(track)
        if len(track) == 0:
            pass
        else:
            mean = np.mean(bunchi[0])*1e3
            inc_qlst_m.append([mean,length]) # this will be the y axis of the graphic
            for lst in track:
                lost_turn, lost_pos = lst
                losttur_m.append([lost_turn, mean])
                lostpos_m.append([lost_pos, mean])
    set_vchamber_h(params, params.scraper_width0) # after calling this function, vchamber's height will be restored

    return lostpos_n, lostpos_m, losttur_n, losttur_m, inc_qlst_n, inc_qlst_m


def p_sim(lostpos_n, lostpos_m, losttur_n, losttur_m, inc_qlst_n, inc_qlst_m, spos):
    fig1, (a1n,a2n,a3n) = plt.subplots(nrows=1,ncols=3, sharey=True, figsize=(10,5))

    # n_coln = np.intp(lostpos_n[:,0])
    # n_colm = np.intp(lostpos_m[:,0])

    fig1.suptitle('tracking without scraper')

    for iten in inc_qlst_n:
        a1n.plot(iten[1], iten[0] , '.', color='blue')
    a1n.set_xlabel(r'Number of electrons lost')
    a1n.set_ylabel(r'position mean [mm]')
    # a1n.set_ylim(0, 12)

    for iten in losttur_n:
        a2n.plot(iten[0], iten[1], '.', color='blue')
    a2n.set_xlabel(r'nturns')
    for idx,iten in enumerate(lostpos_n):
        a3n.plot(spos[iten[0]], iten[1], '.', color='blue')
    a3n.set_xlabel(r'spos [m]')
    fig2, (a1m,a2m,a3m) = plt.subplots(nrows=1,ncols=3, sharey=True, figsize=(10,5))

    fig2.suptitle('tracking with scraper')

    for iten in inc_qlst_m:
        a1m.plot(iten[1], iten[0] , '.', color='blue')
    a1m.set_xlabel(r'Number of electrons lost')
    a1m.set_ylabel(r'position mean [mm]')
    # a1m.set_ylim(0, 12)

    for iten in losttur_m:
        a2m.plot(iten[0], iten[1], '.', color='blue')
    a2m.set_xlabel(r'nturns')
    for iten in lostpos_m:
        a3m.plot(spos[iten[0]], iten[1], '.', color='blue')
    a3m.set_xlabel(r'spos [m]')
    plt.show()

# Parameters

In [3]:
class Params:
    scraper_name = 'SHVC'
    
    # create model
    fitm = pymodels.si.create_accelerator()
    fitm = pymodels.si.fitted_models.vertical_dispersion_and_coupling(fitm)
    fitm.vchamber_on = True
    fitm.radiation_on = True
    fitm.cavity_on = True
    print(fitm)
    
    booster = pymodels.bo.create_accelerator()
    booster.energy = 3e9
    booster[187].voltage = 1e6
    booster.radiation_on=True
    booster.cavity_on=True
    
    # scraper data
    scraper_indices = pyaccel.lattice.find_indices(fitm, 'fam_name', scraper_name)
    scraper_width0 = pyaccel.lattice.get_attribute(fitm, 'hmax', indices=scraper_indices)[0]

    # calc optics
    spos = pyaccel.lattice.find_spos(fitm, indices='closed')
    twiss,*_ = pyaccel.optics.calc_twiss(fitm, indices='closed')
    
    # equilibrium parameters
    eqparams = pyaccel.optics.beam_envelope.EqParamsFromBeamEnvelope(booster)
    coup = 1/100
    emitt0 = eqparams.emit1  # nno-coupling model
    h_emitt = 1/(1 + coup) * emitt0
    v_emitt = coup/(1 + coup) * emitt0
    print('hemitt [pm.rad]:', h_emitt * 1e12)
    print('vemitt [pm.rad]:', v_emitt * 1e12)
    sigmae = eqparams.espread0
    bun_len = eqparams.bunlen

    # defining the index 
    fam_name_dict = pyaccel.lattice.find_dict(fitm, 'fam_name')
    nlk_index = fam_name_dict['InjNLKckr'][0] + 1

energy         : 3000000000.0 eV
harmonic_number: 864
cavity_on      : True
radiation_on   : 1
vchamber_on    : True
lattice version: SI_V25_04_v1.17.0
lattice size   : 6549
lattice length : 518.3898999999925 m
hemitt [pm.rad]: 3429.722487983057
vemitt [pm.rad]: 34.297224879830566


# Analysis

In [4]:
nr_particles = 2

# generate gaussian beam
bunch = pyaccel.tracking.generate_bunch(
    n_part=nr_particles, envelope=None, emit1=Params.h_emitt, emit2=Params.v_emitt, sigmae=Params.sigmae,
    sigmas=Params.bun_len, optics=Params.twiss[Params.nlk_index])

bunch[0] -= 0.008

In [13]:
nr_turns = 10
scraper_width = 0.009

coord_idx = 0
coord_amp_rx = 0.08 # [m]
coord_amp_nrpts_rx = 1200

coord_amp_rxneg = -0.02 # [m]

# variable = varying_incmnts(
#     Params, bunch=bunch, nturn=nr_turns,
#     coord_idx=coord_idx, coord_amp=coord_amp_rx, coord_amp_nrpts=coord_amp_nrpts_rx,
#     scraper_width=scraper_width)
# lostpos_n, lostpos_m, losttur_n, losttur_m, inc_qlst_n, inc_qlst_m = variable
# p_sim(lostpos_n, lostpos_m, losttur_n, losttur_m, inc_qlst_n, inc_qlst_m, Params.spos)

# vary bunch coordinates for different scraper widths
# variable_pos = varying_incmnts(
#     Params, bunch=bunch, nturn=nr_turns,
#     coord_idx=coord_idx, coord_amp=coord_amp_rx, coord_amp_nrpts=coord_amp_nrpts_rx,
#     scraper_width=scraper_width)

variable_neg = varying_incmnts(
    Params, bunch=bunch, nturn=nr_turns,
    coord_idx=coord_idx, coord_amp=coord_amp_rxneg, coord_amp_nrpts=coord_amp_nrpts_rx,
    scraper_width=scraper_width)

fam_name   : SHVC 
pass_method: identity_pass 
hmin       : -0.009 m
hmax       : 0.009 m
vmin       : -0.012 m
vmax       : 0.012 m


In [14]:
# lostpos_np,lostpos_mp, losttur_np,losttur_mp, inc_qlst_np, inc_qlst_mp = variable_pos
lostpos_nn,lostpos_mn, losttur_nn,losttur_mn, inc_qlst_nn, inc_qlst_mn = variable_neg
# # tuple_n, tuple_m =variable

# # lostpos_n, losttur_n, inc_qlst_n = tuple_n
# # lostpos_m, losttur_m,inc_qlst_m = tuple_m




In [ ]:
# rlostposn = np.r_[np.array(lostpos_nn),np.array(lostpos_np)]
# rlostturn = np.r_[np.array(losttur_nn),np.array(losttur_np)]
# rincn = np.r_[np.array(inc_qlst_np),np.array(inc_qlst_nn)]

# rlostposm = np.r_[np.array(lostpos_mn),np.array(lostpos_mp)]
# rlostturm = np.r_[np.array(losttur_mn),np.array(losttur_mp)]
# rincm = np.r_[np.array(inc_qlst_mp),np.array(inc_qlst_mn)]
# p_sim(rlostposn, rlostposm, rlostturn, rlostturm, rincn, rincm, Params.spos)

In [15]:
p_sim(lostpos_nn,lostpos_mn, losttur_nn,losttur_mn, inc_qlst_nn, inc_qlst_mn, Params.spos)

In [ ]:
bunch_p = bunch[0]

bunch_p

In [ ]:
# Suponha que você tenha uma matriz de exemplo
matriz = np.array([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9]])

# Suponha que você tenha uma nova coluna que deseja usar para substituir a coluna existente
nova_coluna = np.array([10, 11, 12])

# Especifique o índice da coluna que deseja substituir (0, 1, 2, etc.)
col_index = 1  # Por exemplo, substituir a segunda coluna (índice 1)

# Substitua a coluna da matriz pela nova coluna
matriz[:, col_index] = nova_coluna

In [ ]:
matriz

In [ ]:

# c = 93
# indices = c_data_treat(variable, c, 0.01,'x')
# plot_tracked(indices, spos, indices, 'x',c, 0.01,100)

In [ ]:
# # anotações que eu pensei que pudessem ser importantes

# # definindo também o modelo fitado para calcular a distribuição de equilíbrio do feixe
# # eu não sei exatamente se essa celula no meu código é util, porque em um primeiro momento vamos variar apenas
# # as posições x, y, x' e y'

# fit_acc = pymodels.si.create_accelerator()
# fit_acc = pymodels.si.fitted_models.vertical_dispersion_and_coupling(fit_acc)
# tousan = tousclass.Tous_analysis(fit_acc)
# deltas = tousan.deltas
# n_turns = tousan.nturns

# orb = pyaccel.tracking.find_orbit6(acc, indices=[0, nlk_index])
# orb = orb[:, 1]
# np.zeros((6, deltas.size)), orb[:, None]

# rin = np.zeros((6, deltas.size))
# rin += orb[:,None]

# mean = np.zeros(6)
# env = pyop.beam_envelope.calc_beamenvelope(fit_acc)
# cov = env[nlk_index]
# particles = np.random.multivariate_normal(mean, cov, size=1).T

# # célula colocada aqui simplesmente para teste de verificação
# # ao executar o código abaixo é possível observar a distribuição gaussiana das posições em x
# plt.figure()
# plt.hist(particles[0], bins=200)
# plt.show()

# # esse procedimento é feito para ir variando os parâmetros de posição 
# vecy = np.linspace(4.8332669321064365e-09, 4.8332669321064365e-02, 1000)
# vecx = np.linspace(2.25702434e-08,2.25702434e-02, 1000)
# # com a função vstack do numpy podemos juntar dois arrays e formar uma matriz
# np.vstack((vecx, vecy))

In [ ]:
# def s_data_treat(l_of_tples, idx, val):
    
#     size = len(l_of_tples)
#     inc = np.linspace(0,val, size)
#     for c, iten in enumerate(l_of_tples):
#         if len(iten) == 0:
#             pass
#         else:
#             j = c
#             break

#     if len(l_of_tples[idx]) == 0:
#         print("all the electrons survived")
#         return

#     else:
#         nt_lost, indices = np.zeros(len(l_of_tples[idx])), np.zeros(len(l_of_tples[idx]))

#         for j,iten in enumerate(l_of_tples[idx]):
#             nt_lostj, idx_lost = iten
#             nt_lost[j] = nt_lostj
#             indices[j] = idx_lost
#         indices = np.intp(indices)
#     ind = np.arange(indices.size) # this index corresponds to how many times the variation of the parameter has been performed

#     return c, inc[c], ind, nt_lost, indices

# def c_data_treat(tup, idx, val, par):

#     '''
#     tup: this tuple contains two lists, these lists are the tracking simulation without and with varying the height of the
#     vchamber from a marker named as horizontal scraper or vertical scraper
#     idx: the variation's index of the parameter 
#     val: variable that goes in numpy linspace, val is the value till the variation must be performed
#     '''

#     all_data = []
#     for iten in tup:
#         resul_tup = s_data_treat(iten, idx, val)

#         c, inc_c, ind, nt_lost, indices = resul_tup
#         all_data.append(indices)
#         print('começa a haver perda de elétrons a partir do incremento {} em {}'.format(inc_c, par))

#     return all_data

# # para montar a função que vai realizar o plot dos gráficos podemos simplesmente passar a função acima em um for

# def plot_tracked(acc, spos, l_all_data, par, idx, val, total): # é bom que essa função receba os valores das lista de indices onde as paticulas foram perdidas

#     '''
#     acc: accelerator
#     spos: s position array along the ring
#     l_all_data: list containing the data that will be analised 
#     par: is a string, corresponding to the variation of the parameter, that will be shown in the title of the graphic/ x, x'
#     idx: is the value that counts wich index was chosen to be analised
#     val: variable that goes in numpy linspace, val is the value till the variation must be performed
#     '''
    
#     nom_track = l_all_data[0] # keep in mind that nominal_tracking is the list of indices that the electrons were lost in the nominal_model with vchamber's height defined by default
#     mod_track = l_all_data[1]
#     inc = np.linspace(0,val, total)

#     ind_nom = np.arange(nom_track.size)
#     ind_mod = np.arange(mod_track.size)

#     fig, (a1, a2) = plt.subplots(nrows= 1, ncols= 2, figsize=(10,5), sharey=True)

#     a1.set_title(r'Without scrapers, varying {}'.format(par), fontsize=16)
#     a2.set_title(r'With scrapers, varying {}'.format(par), fontsize=16)

#     a1.set_xlabel(r's position [m]', fontsize=14)
#     a1.set_ylabel(r'# lost electron', fontsize=14)
    
#     a2.set_xlabel(r's position [m]', fontsize=14)
#     a2.set_ylabel(r'# lost electron', fontsize=14)

#     a1.scatter(spos[nom_track],ind_nom, s=4, label='Value of increment {:.4f} in {}'.format(inc[idx], par))
#     # pyaccel.graphics.draw_lattice(acc, offset=0, height=0, gca=True)
#     a1.legend()
#     print(nom_track)
#     a2.scatter(spos[mod_track],ind_mod, s=4, label='Value of increment {:.4f} in {}'.format(inc[idx], par))
#     a2.legend()
#     # pyaccel.graphics.draw_lattice(acc, offset=0, height=0, gca=True)
#     plt.show()

#     return None